# AWS SageMakerで実行するためのnotebook
* [SageMakerとGitHubの連携方法](https://predictivehacks.com/aws-sagemaker-and-github-integration/)

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator

session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
estimator = Estimator(
    # AWSの用意したdocker image
    # conda環境なのでpipを使うと壊れてしまう可能性がある
    image_uri="763104351884.dkr.ecr.ap-northeast-1.amazonaws.com/pytorch-training:1.12.0-cpu-py38-ubuntu20.04-sagemaker",
    # 自作のdocker image
    role=role,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    code_location="s3://fuyu-bucket/sagemaker/logs", 
    output_path="s3://fuyu-bucket/sagemaker/logs", 
    sagemaker_session=session,
    entry_point="src/train.py"
)


estimator.fit({
    # 入力データの設定
    # Training　Jobではバケット名以降が/opt/ml/input/data/~にコピーされる
    "wallpaper": "s3://fuyu-bucket/wallpaper/"
})